In [1]:
import os
import time
import requests
from selenium import webdriver

In [2]:
# How to execute this code :
# Step 1 : pip install selenium, pillow, requests
# Step 2 : Download the web D=driver that suits the particular web browser
# Step 3 : Put it inside the same folder of the code.

In [3]:
def fetch_image_urls(query:str,min_links_to_fetch:int,wd:webdriver,sleep_between_interactions:int = 1):
    def scroll_to_end(wd):
        wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(sleep_between_interactions)
        
        # build the google query
        
    search_url = 'https://www.google.com/search?q={q}&safe=active&client=firefox-b-d&sxsrf=ALeKk034IvrisourJmJH2XESrBzpzJo81A:1593513272603&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiGwbvEq6nqAhUrzzgGHXECBeoQ_AUoA3oECB8QBQ'
    
#     Load the Page
    wd.get(search_url.format(q=query))
    
    image_urls = set()
    image_count = 0
    res_start = 0
    while image_count < min_links_to_fetch:
        scroll_to_end(wd)
        
        #Get all the thumbnails results
        thumbnail_results = wd.find_elements_by_css_selector('img.Q4LuWd')
        num_res = len(thumbnail_results)
        
        print(f'Found {num_res} search results. Extracting links from {res_start}:{num_res}')
        
        for img in thumbnail_results[res_start:num_res]:
#             try to click every thumbnail so that we get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue
            
            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            c=0
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src') and c<=min_links_to_fetch:
                    image_urls.add(actual_image.get_attribute('src'))
                    c+=1
                    
        image_count = len(image_urls)

        if len(image_urls)>=min_links_to_fetch:
            print(f'Found {len(image_urls)} image links!')
            break
        else:   # If the number of images are not fulfilled because of URL missing.
            print(f'Found {len(image_urls)} image links!')
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector('.mye4qd')
            if load_more_button:
                wd.exceute_script("document.querySelector('.mye4qd').click()")

        res_start = len(thumbnail_results)
    return image_urls

In [4]:
def persist_image(folder_path:str,url:str,counter,search_term:str):
    try:
        image_content = requests.get(url).content
    
    except Exception as e:
        print(f'ERROR - Could not download {url} - {e}')
    
    try:
        f = open(os.path.join(folder_path,search_term+'_'+str(counter)+ '.jpg'),'wb')
        f.write(image_content)
        f.close()
        print(f'SUCCESS - saved {url} - as {folder_path}')
    except Exception as e :
        print(f'ERROR - Could not save {url} - {e}')

In [5]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_of_images=10):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' '))) #Make folder name inside images
    
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)  # Make directory using the target path if its doesn't already exists
    
    with webdriver.Firefox(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term,number_of_images,wd=wd,sleep_between_interactions=0.5)
        
    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter, search_term)
        counter+=1

In [6]:
DRIVER_PATH = './geckodriver'
search_term = input()
# Num of images by default is 10 if not passed
search_and_download(search_term=search_term,driver_path=DRIVER_PATH)

car
Found 100 search results. Extracting links from 0:100
Found 80 image links!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRmCaLlNPM88kaM3H1KNmKk8KDtlKOyrE5psw&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRTEzm7uPo7Xn4VdHtskfSxavod4-LdgBH7dw&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSIExC-Ch46zeei7x1pGAL1X7yY3kXk7oJ2GA&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRxVHWKeWG1ARQi3tO38CP1XsumBLcKFBqfKw&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQsUT4xvnO3Jne4c8HlK-g59UNFBhm3rZnGfg&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQhhTgpAPeEgFScWLDWgDh8U4PBmpWk7cBNeA&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-R62SzktDmz2U6jJax7ANDh_BEyH5BE

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSuuCEyZl4_ilH1ImTZ2iHZk7EfTCtpU_eecQ&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS-j0qP0MkaBaGkiRV7mwBjdLM1GMl-nuUlYA&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTIl2Wb2r6mv2IqF-xPryW0xNv9_EHAHda4fw&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcREXj3ANTm4XLfczENRqO3IcjoozBWOtrAYSg&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ83Bd7nyKj1f55weD5YODBoE3KROLADGDlRQ&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSkwTpW3IQxpeC85SblW4iz2kwnamRXJkIGXg&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTKW2MUPbaTYplm-jGym0FaIZRmuFgGR_6lwQ&usqp=CAU - as ./images\car
SUCCESS - saved https://encrypted-tbn0.gstatic.c